In [157]:
import os
from copy import copy

from utility_dot_py.utility import build_dict_from_csv
from assistments_workbench.config_reader import config

In [158]:
id_file = os.path.join(config.get('localfiles', 'data_path'), 'assistment-sri id mapping for Neil.csv')

In [159]:
id_data = build_dict_from_csv(id_file, 1, True, True)

In [160]:
len(id_data)

1695

In [161]:
nova_file = os.path.join(config.get('localfiles', 'data_path'), 'TerraNova students 11-4-2015.csv')

In [162]:
nova_data = build_dict_from_csv(nova_file, 8, True, False)

In [163]:
len(nova_data)

3421

In [164]:
teacher_file = os.path.join(config.get('localfiles', 'data_path'), 'Teacher_User_Mapping - Sheet1.csv')

In [165]:
teacher_data = build_dict_from_csv(teacher_file, 1, True, False)

In [166]:
len(teacher_data)

45

In [167]:
output_data = []

In [168]:
for sri_id, v in nova_data.items():
    user_ids = id_data.get(sri_id, [{'ASSISTments user id': 'null'}])
    for i in user_ids:
        new_score_data = copy(v)
        new_score_data['student_user_id'] = i['ASSISTments user id']
        new_score_data['teacher_user_id'] = teacher_data.get(v['Teacher ID'], {'user_id': 'null'})['user_id']
        new_score_data['student_sri_id'] = sri_id
        output_data.append(new_score_data)    

In [169]:
len(output_data)

3514

In [170]:
output_data[:3]

[{'Class ID': 'B',
  'Cohort': '2',
  'Condition': 'Control',
  'Grade': '7',
  'Pair': '4d',
  'SPED': 'no',
  'School': 'Westbrook Middle School',
  'School ID': '36',
  'Teacher ID': '140',
  'Teacher Name Last': 'Makin',
  'TerraNova Math Performance Level': '3',
  'TerraNova Score': '703',
  'TerraNova Test Completed': 'yes',
  'student_sri_id': '13356',
  'student_user_id': 'null',
  'teacher_user_id': 'null'},
 {'Class ID': 'C',
  'Cohort': '2',
  'Condition': 'Control',
  'Grade': '7',
  'Pair': '4f',
  'SPED': 'no',
  'School': 'Wiscassett',
  'School ID': '38',
  'Teacher ID': '143',
  'Teacher Name Last': 'Hammond',
  'TerraNova Math Performance Level': '1',
  'TerraNova Score': '487',
  'TerraNova Test Completed': 'yes',
  'student_sri_id': '12861',
  'student_user_id': 'null',
  'teacher_user_id': 'null'},
 {'Class ID': 'B',
  'Cohort': '1',
  'Condition': 'Treatment',
  'Grade': '7',
  'Pair': '1e',
  'SPED': 'no',
  'School': 'Loranger Middle School',
  'School ID': '45'

In [171]:
sql_str = '''INSERT INTO efficacy_score(
            class_id, cohort, condition, grade, pair, sped, school, school_id, 
            teacher_sri_id, teacher_name_last, terranova_math_performance_level, 
            terranova_score, terranova_test_completed, student_user_id, student_sri_id, teacher_user_id)
    VALUES ('%s', %s, '%s', %s, '%s', '%s', '%s', %s, 
            %s, '%s', %s, 
            %s, '%s', %s, 
            %s, %s);\n'''

In [172]:
output_file_name = os.path.join(config.get('localfiles', 'data_path'), 'insert_data_FOR_TEST1_ONLY.sql')

In [173]:
output_file = open(output_file_name, 'w')

In [174]:
for i in output_data:
    for k,v in i.items():
        if len(v) == 0:
            i[k] = 'null'
    outptu_str = sql_str % (i['Class ID'], i['Cohort'], i['Condition'], i['Grade'], i['Pair'], i['SPED'], i['School'], i['School ID'], 
                            i['Teacher ID'], i['Teacher Name Last'], i['TerraNova Math Performance Level'], 
                            i['TerraNova Score'], i['TerraNova Test Completed'], i['student_user_id'], 
                            i['student_sri_id'], i['teacher_user_id'])
        
    output_file.write(outptu_str)

In [175]:
output_file.close()